In [135]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss,accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression, Lasso,Ridge

In [136]:
train = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\train.csv')
test = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\test.csv')
submit = pd.read_csv('C:\\Users\\odoru\\SIGNATE_time_deposit_account\\head_submit_sample.csv')

In [137]:
# jobのunknownの処理
test.replace({'job': {'unknown':'blue-collar'}}, inplace=True)

In [138]:
#score列の作成
train['score'] = np.zeros(train.shape[0])


#個人ローン
#学習データ
loan_none = train.loc[train['loan'] == 'no']
index = loan_none.index.values
index = list(index)

for i in index:
    train.iloc[i,18] = 1
    


#住宅ローン
housing_none = train.loc[train['housing'] == 'no']
index = housing_none.index.values
index = list(index)

score_values = housing_none['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    train.loc[train['housing']=='no','score'] = i



#債務不履行
default = train.loc[train['default'] == 'yes']


score_values = default['score'].values
score_values = score_values - 3
score_values = list(score_values)

for i in score_values:
    train.loc[train['default']=='yes','score'] = i





#経営者
management = train.loc[train['job'] == 'management']


score_values = management['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    train.loc[train['job']=='management','score'] = i
    


#管理者
admin = train.loc[train['job'] == 'admin.']


score_values = admin['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    train.loc[train['job']=='admin.','score'] = i
    



#学生
student = train.loc[train['job'] == 'student']


score_values = student['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    train.loc[train['job']=='student','score'] = i
    




#失業者
unemployed = train.loc[train['job'] == 'unemployed']


score_values = unemployed['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    train.loc[train['job']=='unemployed','score'] = i
    
train.loc[train['job'] == 'unemployed']




#定年退職者
retired = train.loc[train['job'] == 'retired']


score_values = retired['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    train.loc[train['job']=='retired','score'] = i

In [139]:
#テストデータ
#score列の作成
test['score'] = np.zeros(test.shape[0])

#個人ローン
loan_none = test.loc[test['loan'] == 'no']
index = loan_none.index.values
index = list(index)

for i in index:
    test.iloc[i,17] = 1



#住宅ローン
housing_none = test.loc[test['housing'] == 'no']

score_values = housing_none['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    test.loc[test['housing']=='no','score'] = i



#債務不履行
default = test.loc[test['default'] == 'yes']


score_values = default['score'].values
score_values = score_values - 3
score_values = list(score_values)

for i in score_values:
    test.loc[test['default']=='yes','score'] = i





#経営者
management = test.loc[test['job'] == 'management']


score_values = management['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    test.loc[test['job']=='management','score'] = i
    


#管理者
admin = test.loc[test['job'] == 'admin.']


score_values = admin['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    test.loc[test['job']=='admin.','score'] = i
    



#学生
student = test.loc[test['job'] == 'student']


score_values = student['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    test.loc[test['job']=='student','score'] = i
    




#失業者
unemployed = test.loc[test['job'] == 'unemployed']


score_values = unemployed['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    test.loc[test['job']=='unemployed','score'] = i
    



#定年退職者
retired = test.loc[test['job'] == 'retired']


score_values = retired['score'].values
score_values = score_values + 1
score_values = list(score_values)

for i in score_values:
    test.loc[test['job']=='retired','score'] = i
    


In [140]:
cat_cols = ['job','marital','education','default','housing','loan','contact','month']
#LabelEncording
for c in cat_cols:
    le = LabelEncoder()
    le.fit(train[c])
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])

In [141]:
#unknownとunknownでない奴の分割
un_df =  train.loc[train['poutcome'] == 'unknown']
df = train.loc[train['poutcome'] != 'unknown']

In [142]:
un_df.drop(['poutcome','y'], axis = 1, inplace =True)

C:\Users\odoru\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [143]:
df.drop(['y'], axis = 1, inplace =True)

In [144]:
un_df =  train.loc[train['poutcome'] == 'unknown']
df = train.loc[train['poutcome'] != 'unknown']

un_df.drop(['poutcome','y'], axis = 1, inplace =True)
df = df.drop(['y'], axis = 1)

df_poutcome = pd.get_dummies(df['poutcome'])
df = df.drop(['poutcome'], axis = 1)
df = pd.concat([df,df_poutcome], axis=1)

In [145]:
df

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,score,failure,other,success
0,0,31,7,1,1,0,12294,1,0,0,21,8,101,3,498,0,1.0,0,1,0
2,2,35,4,1,2,0,12252,1,0,0,11,8,351,1,826,0,2.0,1,0,0
3,3,31,9,1,1,0,99121,1,1,2,16,7,658,2,120,0,0.0,1,0,0
8,8,32,0,2,1,0,7958,0,0,0,28,10,101,1,525,0,2.0,0,0,1
16,16,31,7,1,1,0,37071,1,0,0,12,7,523,2,831,1,1.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27041,27041,32,7,1,1,0,89447,1,0,0,28,7,207,2,460,0,1.0,1,0,0
27043,27043,31,7,1,1,0,81059,1,0,0,20,0,101,3,148,0,1.0,1,0,0
27060,27060,37,0,1,1,0,42553,0,0,0,12,0,197,1,546,2,2.0,0,0,1
27076,27076,32,9,2,1,0,85647,0,0,0,28,3,101,4,144,1,2.0,1,0,0


In [146]:
df_x

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,score
0,31,7,1,1,0,12294,1,0,0,21,8,101,3,498,0,1.0
2,35,4,1,2,0,12252,1,0,0,11,8,351,1,826,0,2.0
3,31,9,1,1,0,99121,1,1,2,16,7,658,2,120,0,0.0
8,32,0,2,1,0,7958,0,0,0,28,10,101,1,525,0,2.0
16,31,7,1,1,0,37071,1,0,0,12,7,523,2,831,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27041,32,7,1,1,0,89447,1,0,0,28,7,207,2,460,0,1.0
27043,31,7,1,1,0,81059,1,0,0,20,0,101,3,148,0,1.0
27060,37,0,1,1,0,42553,0,0,0,12,0,197,1,546,2,2.0
27076,32,9,2,1,0,85647,0,0,0,28,3,101,4,144,1,2.0


In [147]:
#　カラム一つのやつ
le = LabelEncoder()
df_x = df.iloc[:,1:16]
df_x['score'] = df['score']
df_y_f = df['failure']
df_y_o = df['other']
df_y_s = df['success']
df_test_x = un_df.drop(['id'] ,axis = 1)

In [160]:
#hold out 法
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(df_x))[0]
tr_x,va_x = df_x.iloc[tr_idx], df_x.iloc[va_idx]
tr_y,va_y = df_y_s.iloc[tr_idx], df_y_s.iloc[va_idx] 

clf = Ridge()
clf.fit(tr_x, tr_y)
va_pred = clf.predict(va_x)
score = log_loss(va_y,va_pred)
print(score)


0.26932602356546975


In [151]:
#failureのやつ
f = np.where(va_pred> 0.5, 1,0 )
print(f)
accuracy = accuracy_score(va_y, f)
print(accuracy)

[1 1 1 ... 0 1 1]
0.7122877122877123


In [153]:
o = np.where(va_pred> 0.7, 1,0 )
print(o)
accuracy = accuracy_score(va_y, o)
print(accuracy)

[0 0 0 ... 0 0 0]
0.8061938061938062


In [155]:
s = np.where(va_pred> 0.9 ,1,0 )
print(s)
accuracy = accuracy_score(va_y, s)
print(accuracy)

[0 0 0 ... 0 0 0]
0.8961038961038961


In [157]:
test_pred = clf.predict(df_test_x)
f =  np.where(test_pred> 0.5, 1,0 )

In [159]:
test_pred = clf.predict(df_test_x)
o =  np.where(test_pred> 0.7, 1,0 )

In [161]:
test_pred = clf.predict(df_test_x)
s =  np.where(test_pred> 0.9, 1,0 )

In [165]:
df_test_x['failure'] = f

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,score,failure
1,29,2,2,2,0,43027,0,0,0,22,1,158,2,702,0,2.0,1
4,48,10,1,0,0,42005,1,0,1,3,0,177,1,273,0,3.0,1
5,30,1,1,0,0,-6160,0,0,0,18,1,158,1,632,0,2.0,1
6,33,1,1,1,0,88282,1,0,0,27,7,345,1,272,0,1.0,1
7,30,0,2,1,0,59830,1,0,0,27,7,184,1,639,0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27095,37,1,1,1,0,26661,1,0,0,27,7,345,4,425,0,1.0,1
27096,35,7,1,1,0,42150,1,0,0,27,7,121,1,719,0,1.0,1
27097,35,7,1,3,0,34531,0,0,0,28,5,177,2,121,0,2.0,1
27098,30,0,2,1,0,99621,1,0,0,27,7,121,1,100,0,2.0,1


In [166]:
df_test_x['other'] = o
df_test_x['success'] = s

In [174]:
a = pd.get_dummies(df_y)
a.columns = ['failure','other','success']
b = pd.concat([df_x,a],axis=1)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,score,failure,other,success
0,31,7,1,1,0,12294,1,0,0,21,8,101,3,498,0,1.0,0,1,0
2,35,4,1,2,0,12252,1,0,0,11,8,351,1,826,0,2.0,1,0,0
3,31,9,1,1,0,99121,1,1,2,16,7,658,2,120,0,0.0,1,0,0
8,32,0,2,1,0,7958,0,0,0,28,10,101,1,525,0,2.0,0,0,1
16,31,7,1,1,0,37071,1,0,0,12,7,523,2,831,1,1.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27041,32,7,1,1,0,89447,1,0,0,28,7,207,2,460,0,1.0,1,0,0
27043,31,7,1,1,0,81059,1,0,0,20,0,101,3,148,0,1.0,1,0,0
27060,37,0,1,1,0,42553,0,0,0,12,0,197,1,546,2,2.0,0,0,1
27076,32,9,2,1,0,85647,0,0,0,28,3,101,4,144,1,2.0,1,0,0


In [179]:
c = pd.concat([b,df_test_x], axis=0)

In [184]:
f_df = c.sort_index()
f_df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,score,failure,other,success
0,31,7,1,1,0,12294,1,0,0,21,8,101,3,498,0,1.0,0,1,0
1,29,2,2,2,0,43027,0,0,0,22,1,158,2,702,0,2.0,1,0,0
2,35,4,1,2,0,12252,1,0,0,11,8,351,1,826,0,2.0,1,0,0
3,31,9,1,1,0,99121,1,1,2,16,7,658,2,120,0,0.0,1,0,0
4,48,10,1,0,0,42005,1,0,1,3,0,177,1,273,0,3.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27095,37,1,1,1,0,26661,1,0,0,27,7,345,4,425,0,1.0,1,0,0
27096,35,7,1,1,0,42150,1,0,0,27,7,121,1,719,0,1.0,1,0,0
27097,35,7,1,3,0,34531,0,0,0,28,5,177,2,121,0,2.0,1,0,0
27098,30,0,2,1,0,99621,1,0,0,27,7,121,1,100,0,2.0,1,0,0


In [186]:
d = f_df.loc[:,['failure','other','success']]

In [188]:
d.to_csv('logistic_poutcome.csv',index=False)